In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflowjs as tfjss
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2

# Path ke folder dataset
dataset_folder = 'images/'

# Path ke file styles.csv
styles_file = 'styles.csv'

# Membaca data dari file styles.csv
data = pd.read_csv(styles_file)

# Membaca gambar dan label dari folder dataset
images = []


# Memuat label ke dalam variabel
label_csv = data['articleType']


# Mengonversi label FashionImage menjadi tipe data string
label_csv = label_csv.astype(str)

# Menggabungkan gambar dari folder 'images' dengan dataset MNIST
image_folder = 'images/'

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Mengubah gambar menjadi skala abu-abu (grayscale)
    image = cv2.resize(image, (28, 28))  # Menyesuaikan dimensi gambar menjadi 28x28
    image = np.expand_dims(image, axis=-1)  # Menambahkan dimensi kanal (1) pada gambar
    images.append(image)

# Konversi data gambar menjadi array numpy
images_data = np.array(images)

In [ ]:
# Mengubah label menjadi angka menggunakan LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(label_csv)
num_classes = len(label_encoder.classes_)

#mengetahui jumlah data
print('jumlah gambar ', images_data.shape)
print('jumlah label ', label_csv.shape)


In [ ]:
# Membagi dataset menjadi data training, data test, dan data validasi
x_train, x_test, y_train, y_test = train_test_split(images_data, labels, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

# Normalisasi pixel gambar menjadi rentang [0, 1]
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.85:
            print("\nReached 85% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# Membangun arsitektur model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
]) 

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Pelatihan model
model.fit(x_train, y_train, batch_size=64, epochs=30, validation_data=(x_val, y_val), callbacks=[callbacks])

# Evaluasi model pada data test
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)


In [ ]:
# Path ke gambar yang akan diprediksi
image_path = 'CV.jpg'

# Membaca gambar
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
image = cv2.resize(image, (28, 28))
image = np.expand_dims(image, axis=-1)  
image = image.astype('float32') / 255

# Melakukan prediksi
predictions = model.predict(np.array([image]))
predicted_class = np.argmax(predictions)

# Mengonversi kelas prediksi menjadi label
predicted_label = label_encoder.classes_[predicted_class]

# Menampilkan hasil prediksi
print('Predicted Label:', predicted_label)


In [ ]:
# Path untuk menyimpan model TensorFlow.js
tfjs_model_path = 'tfjs_model' #Konversi ke KSON

# Konversi model ke format TensorFlow.js
tfjs.converters.save_keras_model(model, tfjs_model_path)

# Simpan model
model.save('image_classification_model.h5')
